In [1]:
import pandas as pd
import random

cities_df = pd.read_csv('cities.tsv', sep='\t', header=None)
cities_df.columns = ['name', 'interest', 'longitude', 'latitude']


routes_df = pd.read_csv('routes.tsv', sep='\t', header=None)
routes_df.columns = ['from_city', 'to_city', 'travel_time', 'cost']

print(cities_df.head())
print(routes_df.head())

        name  interest     longitude      latitude
0   Alicante  0.602388 -5.434292e+04  4.628085e+06
1  Amsterdam  0.726813  5.447535e+05  6.867808e+06
2     Ancona  0.598182  1.504716e+06  5.406364e+06
3    Antalya  0.598325  3.417006e+06  4.425311e+06
4     Athens  0.732723  2.641423e+06  4.577157e+06
  from_city     to_city  travel_time    cost
0  Alicante   Amsterdam        10500   28.99
1  Alicante        Bari         8400  133.99
2  Alicante     Belfast        11400   99.99
3  Alicante  Birmingham         9900   58.99
4  Alicante    Bordeaux         5700   70.99


In [ ]:
def get_possible_routes(current_city, total, limits):
    possible_routes = routes_df[
        (routes_df['from_city'] == current_city) &
        (routes_df['travel_time'] + total['time'] <= limits['time']) &
        (routes_df['cost'] + total['cost'] <= limits['cost'])
    ]

    return possible_routes

In [13]:
def get_route(time_limit, cost_limit, next_route_func):
    current_city = random.choice(routes_df['from_city'].unique())
    route = [current_city]
    visited_cities = set(route)
    total = {
        "time": 0,
        "cost": 0
    }
    limits = {
        "time": time_limit,
        "cost": cost_limit
    }
    interest_of_cities = []
    while True:
        possible_routes = get_possible_routes(current_city, total, limits)
        if possible_routes.empty:
            break

        next_route = next_route_func(possible_routes, visited_cities)
        
        current_city = next_route['to_city']
        if current_city not in visited_cities:
            interest_of_cities.append(cities_df[cities_df['name'] == current_city]['interest'].values[0])

        visited_cities = set(route)
        route.append(current_city)
        total['time'] += next_route['travel_time']
        total['cost'] += next_route['cost']

    print("visited cities", visited_cities)
    print("interest of cities array", interest_of_cities)
    print("Route:", " -> ".join(route))
    print("Total Travel Time:", total['time'])
    print("Total Cost:", total['cost'])

    return interest_of_cities

In [20]:
cost_limits = [50, 200, 500, 1000, 10000]
time_limits = [3, 5, 10, 20, 100]
results_df = pd.DataFrame(index=[cost_limits], columns=[time_limits])

def get_results(next_route_func, N=100):
    for cost_limit in cost_limits:
        for time_limit in time_limits:
            interest_scores = []
            for _ in range(N):
                time_limit_s = time_limit * 3600
                interest_of_cities = get_route(time_limit_s, cost_limit, next_route_func)
                interest_scores.append(sum(interest_of_cities))

            avg = sum(interest_scores) / len(interest_scores)
            std_deviation = (sum((x - avg) ** 2 for x in interest_scores) / len(interest_scores)) ** 0.5
            results_df.loc[cost_limit, time_limit] = f"avg: {avg:.2f}, std: {std_deviation:.2f}"

    return results_df

def random_route(possible_routes, _):
    return possible_routes.sample(1).iloc[0]

random_res = get_results(random_route)
random_res.to_csv('random_res.csv')

visited cities {'Genoa'}
interest of cities array [np.float64(0.8307827616237295)]
Route: Genoa -> London
Total Travel Time: 7800
Total Cost: 37.99
visited cities {'Stockholm', 'Malmoe'}
interest of cities array [np.float64(0.740175301943447), np.float64(0.6645405979969531)]
Route: Malmoe -> Stockholm -> Poznan
Total Travel Time: 10800
Total Cost: 47.1174
visited cities {'Lille'}
interest of cities array [np.float64(0.6682142268493704)]
Route: Lille -> Toulouse
Total Travel Time: 5400
Total Cost: 40.15
visited cities {'Katowice', 'Milan'}
interest of cities array [np.float64(0.7390652078701349), np.float64(0.694769776448509)]
Route: Katowice -> Milan -> Zagreb
Total Travel Time: 10800
Total Cost: 30.1462
visited cities {'Madrid'}
interest of cities array [np.float64(0.7068062387577692)]
Route: Madrid -> Turin
Total Travel Time: 7500
Total Cost: 28.99
visited cities {'Tallinn'}
interest of cities array [np.float64(0.6614009527445156)]
Route: Tallinn -> Nuernberg
Total Travel Time: 8700


In [5]:
def get_travel_time(df, from_city, to_city):
    return df[
        (df['from_city'] == from_city) & 
        (df['to_city'] == to_city)
    ]['travel_time'].values[0]

def get_cost(df, from_city, to_city):
    return df[
        (df['from_city'] == from_city) & 
        (df['to_city'] == to_city)
    ]['cost'].values[0]

In [19]:
from sklearn.preprocessing import StandardScaler

standard_scaler = StandardScaler()

std_city_df = pd.DataFrame()
std_city_df['name'] = cities_df['name']
std_city_df['interest'] = standard_scaler.fit_transform(cities_df[['interest']]) 

std_routes_df = pd.DataFrame()
std_routes_df['from_city'] = routes_df['from_city']
std_routes_df['to_city'] = routes_df['to_city']
std_routes_df['travel_time'] = standard_scaler.fit_transform(routes_df[['travel_time']])
std_routes_df['cost'] = standard_scaler.fit_transform(routes_df[['cost']])
std_city_df = std_city_df.set_index('name')

std_routes_df['greedy_score'] = std_routes_df.apply(
    lambda row: std_city_df.loc[row['to_city'], 'interest'] - row['travel_time'] - row['cost'], axis=1
)

def find_best_greedy_route(possible_routes, visited_cities):
    merged_df = possible_routes.merge(
        std_routes_df[['from_city', 'to_city', 'greedy_score']],
        on=['from_city', 'to_city'],
        how='inner',
    )

    filtered_merged_df = merged_df[~merged_df['to_city'].isin(visited_cities)]

    max_greedy_idx = None
    if not filtered_merged_df.empty:
        merged_df = filtered_merged_df
        max_greedy_idx = merged_df['greedy_score'].idxmax()
    else:
        max_greedy_idx = merged_df.sample(1).index[0]
    
    max_row = merged_df.loc[max_greedy_idx]

    return possible_routes[
        (possible_routes['from_city'] == max_row['from_city']) & 
        (possible_routes['to_city'] == max_row['to_city'])
    ].iloc[0]

greedy_res = get_results(find_best_greedy_route)
greedy_res.to_csv('greedy_res.csv')

visited cities {'Karlsruhe', 'London'}
interest of cities array [np.float64(0.8307827616237295), np.float64(0.7006323589104861)]
Route: Karlsruhe -> London -> Cologne
Total Travel Time: 9600
Total Cost: 49.7938
visited cities {'Karlsruhe', 'London'}
interest of cities array [np.float64(0.8307827616237295), np.float64(0.7006323589104861)]
Route: Karlsruhe -> London -> Cologne
Total Travel Time: 9600
Total Cost: 49.7938
visited cities {'Lublin', 'Gdansk'}
interest of cities array [np.float64(0.658692247236834), np.float64(0.769412325825045)]
Route: Lublin -> Gdansk -> Vienna
Total Travel Time: 9300
Total Cost: 23.6498
visited cities {'Bratislava', 'Rome'}
interest of cities array [np.float64(0.7896107180689524), np.float64(0.6561148171447915)]
Route: Bratislava -> Rome -> Palermo
Total Travel Time: 9900
Total Cost: 45.76
visited cities {'Geneva'}
interest of cities array [np.float64(0.8317101715588673)]
Route: Geneva -> Paris
Total Travel Time: 4200
Total Cost: 39.28
visited cities {'Mil

In [27]:
import numpy as np
import time

def exploit(q_table_df, current_city, possible_routes):
    if q_table_df.loc[current_city, possible_routes['to_city']].max() == 0:
        return possible_routes.sample(1).iloc[0]['to_city']

    return  q_table_df.loc[current_city, possible_routes['to_city']].idxmax()
    

def q_learning(config, limits):
    q_table_df = pd.DataFrame(np.zeros((len(cities_df), len(cities_df))), index=cities_df['name'], columns=cities_df['name'])
    alpha = config['learning_rate']
    gamma = config['discount_factor']

    for _ in range(config['num_episodes']):
        current_city = random.choice(cities_df['name'])
        total_time = 0
        total_cost = 0
        epsilon = config['exploration_rate']
        momentum = config['exploration_momentum']
        # visited_cities = []
        # print("Episode:", _)

        while True:
            possible_routes = get_possible_routes(current_city, {'time': total_time, 'cost': total_cost}, limits)
            # possible_routes = possible_routes[~possible_routes['to_city'].isin(visited_cities)]
            if possible_routes.empty:
                break

            if np.random.uniform(0, 1) < epsilon:
                next_city = possible_routes.sample(1).iloc[0]['to_city']
            else:
                next_city = exploit(q_table_df, current_city, possible_routes)
            
            # if next_city not in visited_cities:
            #     visited_cities.append(next_city)

            interest = cities_df[cities_df['name'] == next_city]['interest'].values[0]
            time = get_travel_time(routes_df, current_city, next_city)
            cost = get_cost(routes_df, current_city, next_city)
            total_time += time
            total_cost += cost 

            possible_routes_for_next_city = get_possible_routes(next_city, {'time': total_time, 'cost': total_cost}, limits)
            if possible_routes_for_next_city.empty:
                break

            next_max_q_score = q_table_df.loc[next_city, possible_routes_for_next_city['to_city']].max()
            curr_max_q_score = q_table_df.loc[current_city, next_city]
            
            new_value = (1 - alpha) * curr_max_q_score + alpha * (interest + gamma * next_max_q_score)
            # print('setting new value', new_value, 'for', current_city, next_city)
            q_table_df.loc[current_city, next_city] = new_value

            current_city = next_city
            epsilon = epsilon * momentum

        
    return q_table_df

qconfig = {
    "learning_rate": 0.1,
    "discount_factor": 0.9,
    "exploration_rate": 0.5,
    "exploration_momentum": 0.9,
    "num_episodes": 1000
}

start_time = time.time()
q_table = q_learning(qconfig, {"cost": 10000, "time": 36000})
end_time = time.time()

print(f"Q-learning execution time: {end_time - start_time} seconds")

def find_best_q_route(possible_routes, visited_cities):
    filtered_routes = possible_routes[~possible_routes['to_city'].isin(visited_cities)]
    if not filtered_routes.empty:
        possible_routes = filtered_routes

    best_route = q_table.loc[possible_routes.iloc[0]['from_city'], possible_routes['to_city']].idxmax()
    # print('best route found', possible_routes[possible_routes['to_city'] == best_route].iloc[0])
    # print('possible_routes_were:' , q_table.loc[possible_routes.iloc[0]['from_city'], possible_routes['to_city']])
    return possible_routes[possible_routes['to_city'] == best_route].iloc[0]

q_table_res = get_results(find_best_q_route)
q_table_res.to_csv('q_table_res.csv')

Q-learning execution time: 27.022998094558716 seconds
visited cities {'Copenhagen', 'Gdansk'}
interest of cities array [np.float64(0.658692247236834), np.float64(0.6705586482688143)]
Route: Copenhagen -> Gdansk -> Vilnius
Total Travel Time: 8400
Total Cost: 33.8298
visited cities {'Oslo'}
interest of cities array [np.float64(0.6936604637257605)]
Route: Oslo -> Krakow
Total Travel Time: 7980
Total Cost: 25.2517
visited cities {'Prague'}
interest of cities array [np.float64(0.6767042570635242)]
Route: Prague -> Edinburgh
Total Travel Time: 8700
Total Cost: 48.4498
visited cities {'Gdansk', 'Malmoe'}
interest of cities array [np.float64(0.658692247236834), np.float64(0.71373169654299)]
Route: Malmoe -> Gdansk -> Oslo
Total Travel Time: 9780
Total Cost: 47.873200000000004
visited cities {'Kosice'}
interest of cities array [np.float64(0.7434914632787466)]
Route: Kosice -> Prague
Total Travel Time: 4500
Total Cost: 48.99
visited cities {'Antalya'}
interest of cities array []
Route: Antalya
T